Thursday, June 8, 2023

I was running this in the conda environment nlpwt, but then realized it does not have the gpu version of PyTorch ... wtf!?

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
delimiter = '.'

In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi[delimiter] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


Since we have a GPU, let's use it, shall we ... 

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [7]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] # crop and append

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [8]:
# MLP revisited
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) 
b1 = torch.randn(n_hidden,                        generator=g) 
W2 = torch.randn((n_hidden, vocab_size),          generator=g) 
b2 = torch.randn(vocab_size,                      generator=g) 

parameters = [C, W1, b1, W2, b2]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
    p.requires_grad = True

11897


In [9]:
%%time
# same optimization as last time
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
    
    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
    
    # 1) Forward Pass
    # embed the characters into vectors
    emb = C[Xb] 
    # concatenate the vectors
    embcat = emb.view(emb.shape[0], -1) 
    # hidden layer pre-activation
    hpreact = embcat @ W1 + b1
    # hidden layer
    h = torch.tanh(hpreact)
    # output layer
    logits = h @ W2 + b2
    # loss function
    loss = F.cross_entropy(logits, Yb)
    
    # 2) Backward Pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # 3) update
    # step learning rate decay
    lr = 0.1 if i < 10000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
    
    # 4) track stats
    if i % 10000 == 0:
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())
    
# running on all 8 cpus ... 
# CPU times: user 6min 42s, sys: 5.67 s, total: 6min 48s
# Wall time: 51.6 s
    

      0/ 200000: 27.8817
  10000/ 200000: 2.8331
  20000/ 200000: 2.4988
  30000/ 200000: 2.6304
  40000/ 200000: 2.0544
  50000/ 200000: 2.4894
  60000/ 200000: 2.4372
  70000/ 200000: 2.1040
  80000/ 200000: 2.3070
  90000/ 200000: 2.1245
 100000/ 200000: 1.9729
 110000/ 200000: 2.5353
 120000/ 200000: 2.1414
 130000/ 200000: 2.5304
 140000/ 200000: 2.3557
 150000/ 200000: 2.2075
 160000/ 200000: 2.3027
 170000/ 200000: 1.8006
 180000/ 200000: 2.3113
 190000/ 200000: 2.0539
CPU times: user 7min 1s, sys: 5.89 s, total: 7min 7s
Wall time: 54.4 s
